In [1]:
import numpy as np
#import cv2
import re
from PIL import Image
import os

In [2]:
# choose the corresponding domain: cloudy1.txt, rainy1.txt, sunny1.txt, night1.txt
file = open("cloudy1.txt", 'r')

In [3]:
ind = []
file_names = []
sizes = []
bboxes = []
f = file.readlines()

# remove the empty lines
for i in range(0, len(f)):
    f[i] = f[i].replace('\n', '')
print(len(f))
new_tab = []
for e in f:
    e.replace('\n', '')
    if e != '':
        new_tab.append(e)
print(len(new_tab))

# extract the fields in the corresponding arrays
# n is the number of element in the file
n = 23272
i = 0
elem = 0
while(elem < n):
    ind.append(new_tab[i])
    file_names.append(new_tab[i+1])
    sizes.append(new_tab[i+2])
    bbox = []
    for j in range(0, int(new_tab[i+4])):
        bbox.append(new_tab[i+5+j])
    bboxes.append(bbox)
    elem += 1
    i += (5 + int(new_tab[i+4]))

297224
250678


In [4]:
def print_fields(n):
    print(ind[n])
    print(file_names[n])
    print(sizes[n])
    print(bboxes[n])
    print("")

for i in range(0, 2):
    print_fields(i)

# 1
/video_data_20180803/jig_KYT01/20180713_KYT/dw_2018_07_13_13-47-34_000000_20fps_bae/camera_2_center_fov60.h264/12_00942.png
3 1208 1920 1
['1 508.1 638.5 642.0 729.8', '1 846.0 650.1 898.3 697.6', '1 895.3 599.3 1118.6 808.5', '1 1130.6 646.4 1162.2 675.3', '2 910.8 572.6 934.4 595.2']

# 2
/video_data_20180803/jig_KYT01/20180713_KYT/dw_2018_07_13_13-47-34_000000_20fps_bae/camera_2_center_fov60.h264/12_00201.png
3 1208 1920 1
['1 1566.3 604.6 1918.3 1036.7', '1 1324.0 642.6 1498.2 775.9', '1 1227.7 638.0 1312.8 712.5', '1 1105.7 639.3 1190.8 709.2', '1 1147.2 629.4 1190.8 682.9', '1 1224.4 599.7 1266.6 645.9', '1 1189.4 585.2 1219.1 620.8', '1 1185.5 601.1 1210.5 628.8', '1 1167.7 621.5 1202.6 666.4', '1 749.2 675.6 865.9 731.7', '2 378.8 134.1 489.2 240.7']



In [5]:
file_names[3512]
#s = []
#for word in bboxes[0][0].split():
#    if word.isdigit():
#        s.append(word)
#len(s)

'/video_data_20180809/jig_KYT01/20180727_KYT/dw_2018_07_27_10-56-14_000000_20fps_bae/camera_2_center_fov60.h264/23_01822.png'

In [6]:
# convert the string array bboxes into an int array
def convert_array_string_to_array_int(bboxes):
    bboxes_int = []
    for elem in bboxes:
        bbox = []
        for e in elem:
            res = re.findall(r"[-+]?\d*\.\d+|\d+", e)
            res_int = []
            res_int.append(int(res[0]))
            res_int.append(int(float((res[1]))))
            res_int.append(int(float((res[2]))))
            res_int.append(int(float((res[3]))))
            res_int.append(int(float((res[4]))))
            bbox.append(res_int)
        bboxes_int.append(bbox)
    
    return bboxes_int

bboxes_int = convert_array_string_to_array_int(bboxes)

In [7]:
bboxes_int[0]

[[1, 508, 638, 642, 729],
 [1, 846, 650, 898, 697],
 [1, 895, 599, 1118, 808],
 [1, 1130, 646, 1162, 675],
 [2, 910, 572, 934, 595]]

In [8]:
file_names[0]

'/video_data_20180803/jig_KYT01/20180713_KYT/dw_2018_07_13_13-47-34_000000_20fps_bae/camera_2_center_fov60.h264/12_00942.png'

In [9]:
#for i in range(0, len(file_names)):
#    file_names[i] = os.path.basename(file_names[i])

imgs = []
for root, dirs, files in os.walk('cloudy'):
    for i in files:
        imgs.append(os.path.join(root, i))
#for i in range(0, len(imgs)):
#    imgs[i] = os.path.basename(imgs[i])
print(len(imgs))
print(imgs[10])

12723
cloudy/video_data_20180703/jig_KYT01/20180622_KYT/dw_2018_06_22_13-42-45_000000_20fps_bae/camera_2_center_fov60.h264/7_00224.png


In [10]:
def exctract_object_PIL(img, bbox):
    res = img.crop((bbox[1], bbox[2], bbox[3], bbox[4]))
    
    return res

def extract_object(img, bbox):
    # x_max - x_min
    width = bbox[3] - bbox[1]
    # y_max - y_min
    height = bbox[4] - bbox[2]
    res = img[bbox[1]:bbox[1]+width, bbox[2]:bbox[2]+height]
    
    return res

In [11]:
for i in range(0, len(bboxes_int)):
    if (i % 100) == 0:
        print(i)
    #if os.path.basename(file_names[i]) in imgs and file_names[i].startswith('/video') and os.path.basename(file_names[i]): #!= '0_00049.png':
    if ('cloudy' + file_names[i]) in imgs:
        img = Image.open('cloudy' + file_names[i])
        for j in range(0, len(bboxes_int[i])):
            if not (bboxes_int[i][j][1] == 0 and bboxes_int[i][j][2] == 0 and bboxes_int[i][j][3] == 0 and bboxes_int[i][j][4] == 0):
                obj_img = exctract_object_PIL(img, bboxes_int[i][j])
                if bboxes_int[i][j][0] == 1:
                    obj_img.save("cloudy_cars/"+"bb"+str(j)+"_"+os.path.basename(file_names[i]))
                elif bboxes_int[i][j][0] == 2:
                    obj_img.save("cloudy_signs/"+"bb"+str(j)+"_"+os.path.basename(file_names[i]))
                elif bboxes_int[i][j][0] == 3:
                    obj_img.save("cloudy_people/"+"bb"+str(j)+"_"+os.path.basename(file_names[i]))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18